# L-Alanin with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up.

In [1]:
from xharpy import shelxl_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions, refine, write_cif, write_res, write_fcf
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

In [4]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [5]:
atom_table['adp_type'] = 'Uani'
atom_table[['U_11', 'U_22', 'U_33']] = atom_table[['U_11', 'U_22', 'U_33']].fillna(0.01)
atom_table[['U_12', 'U_23', 'U_13']] = atom_table[['U_12', 'U_23', 'U_13']].fillna(0.00)

In [6]:
hkl = shelxl_hkl_to_pd('iam.hkl')

In [7]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)

In [8]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw')
}

In [9]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [16]:
computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.15,
    'gridinterpolation': 4,
    #'poissonsolver': poissonsolver,d
    'average_symmequiv': False,
    'convergence':{'density': 1e-7},
    #'kpts': {'size': (2, 2, 2), 'gamma': True},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2
}

In [17]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-03 21:19:36.397390
Preparing
  calculating the core structure factor for O
  calculating the core structure factor for N
  calculating the core structure factor for H
  calculating the core structure factor for C
  calculating first atomic form factors
  calculated density with energy -335.9242704581883
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.016988, nit: 2, Optimization terminated successfully.
  minimizing least squares sum
  wR2: 0.016954, nit: 127, Optimization terminated successfully.
step 1: calculating new structure factors
  calculated density with energy -335.92532470322186
  minimizing least squares sum
  wR2: 0.016954, nit: 125, Optimization terminated successfully.
step 2: calculating new structure factors
  calculated density with energy -335.9251425518599
  minimizing least squares sum
  wR2: 0.016954, nit: 126, Optimization terminated successfully.
step 3: calculating new structure factor

In [18]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [19]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [20]:
write_res(
    out_res_path=os.path.join(output_folder, 'xharpy_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [21]:
write_cif(
    output_cif_path=os.path.join(output_folder, 'xharpy.cif'),
    cif_dataset='xharpy',
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [22]:
hkl

,h,k,l,intensity,esd_int,weight,strong_condition
0,0,2,0,183.414597,1.752700,0.097656,True
1,0,4,0,5175.345716,44.359746,0.000152,True
2,0,6,0,89.513685,1.040666,0.277008,True
3,0,8,0,45.263482,0.717512,0.582717,True
4,0,10,0,0.799669,0.339586,2.601457,True
...,...,...,...,...,...,...,...
2514,12,6,3,0.909213,0.405312,1.826150,True
2515,12,0,4,14.465254,0.942076,0.338021,True
2516,12,1,4,44.896511,1.413115,0.150231,True
2517,12,2,4,9.240016,0.848964,0.416233,True
